In [0]:
display(dbutils.fs.ls("/FileStore/tables"))

path,name,size,modificationTime
dbfs:/FileStore/tables/new_user_credentials-1.csv,new_user_credentials-1.csv,202,1655362445000
dbfs:/FileStore/tables/new_user_credentials.csv,new_user_credentials.csv,202,1655362137000


In [0]:
from pyspark.sql.functions import *

import urllib

In [0]:
# Define file type
file_type = "csv"
# Whether the file has a header
first_row_is_header = "true"
# Delimiter used in the file
delimiter = ","
# Read the CSV file to spark dataframe
aws_keys_df = spark.read.format(file_type)\
.option("header", first_row_is_header)\
.option("sep", delimiter)\
.load("/FileStore/tables/new_user_credentials.csv")

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.where(col('User name')=='datatraininguser').select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.where(col('User name')=='datatraininguser').select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "datatraining-parquet"
# Mount name for the bucket
MOUNT_NAME = "/mnt/datatraining-parquet"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)


Out[7]: True

In [0]:
# Check if the AWS S3 bucket was mounted successfully
display(dbutils.fs.ls("/mnt/datatraining-parquet/"))

path,name,size,modificationTime
dbfs:/mnt/datatraining-parquet/yellow_tripdata_2020-04.parquet,yellow_tripdata_2020-04.parquet,4442620,1655357688000
dbfs:/mnt/datatraining-parquet/yellow_tripdata_2020-06.parquet,yellow_tripdata_2020-06.parquet,9505358,1655357689000
dbfs:/mnt/datatraining-parquet/yellow_tripdata_2020-08.parquet,yellow_tripdata_2020-08.parquet,16601463,1655357690000


In [0]:
# Read a dataset in parquet format from S3 to Databricks
file_location = "/mnt/datatraining-parquet/yellow_tripdata_2020-04.parquet"
file_type = "parquet"
df = spark.read.format(file_type).load(file_location)
display(df)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2020-04-01T00:41:22.000+0000,2020-04-01T01:01:53.000+0000,1.0,1.2,1.0,N,41,24,2,5.5,0.5,0.5,0.0,0.0,0.3,6.8,0.0,null
1,2020-04-01T00:56:00.000+0000,2020-04-01T01:09:25.000+0000,1.0,3.4,1.0,N,95,197,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,0.0,null
1,2020-04-01T00:00:26.000+0000,2020-04-01T00:09:25.000+0000,1.0,2.8,1.0,N,237,137,1,10.0,3.0,0.5,1.0,0.0,0.3,14.8,2.5,null
1,2020-04-01T00:24:38.000+0000,2020-04-01T00:34:38.000+0000,0.0,2.6,1.0,N,68,142,1,10.0,3.0,0.5,1.0,0.0,0.3,14.8,2.5,null
2,2020-04-01T00:13:24.000+0000,2020-04-01T00:18:26.000+0000,1.0,1.44,1.0,Y,263,74,1,6.5,0.5,0.5,3.0,0.0,0.3,13.3,2.5,null
2,2020-04-01T00:24:36.000+0000,2020-04-01T00:33:09.000+0000,1.0,2.93,1.0,N,75,170,2,10.5,0.5,0.5,0.0,0.0,0.3,14.3,2.5,null
2,2020-04-01T00:56:56.000+0000,2020-04-01T01:09:13.000+0000,1.0,6.86,1.0,N,141,243,1,20.0,0.5,0.5,4.76,0.0,0.3,28.56,2.5,null
2,2020-04-01T00:06:56.000+0000,2020-04-01T00:14:15.000+0000,1.0,1.19,1.0,N,137,68,2,7.0,0.5,0.5,0.0,0.0,0.3,10.8,2.5,null
1,2020-04-01T00:50:05.000+0000,2020-04-01T01:08:54.000+0000,1.0,11.3,1.0,N,229,185,2,31.5,3.0,0.5,0.0,0.0,0.3,35.3,2.5,null
2,2020-04-01T00:07:10.000+0000,2020-04-01T00:18:45.000+0000,1.0,3.68,1.0,N,140,226,1,13.0,0.5,0.5,3.36,0.0,0.3,20.16,2.5,null


In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "datatraining-csv"
# Mount name for the bucket
MOUNT_NAME = "/mnt/datatraining-csv"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

Out[8]: True

In [0]:
# Check if the AWS S3 bucket was mounted successfully
display(dbutils.fs.ls("/mnt/datatraining-csv/"))

path,name,size,modificationTime
dbfs:/mnt/datatraining-csv/yellow_tripdata_2020-05.csv,yellow_tripdata_2020-05.csv,42846589,1655357688000
dbfs:/mnt/datatraining-csv/yellow_tripdata_2020-07.csv,yellow_tripdata_2020-07.csv,98656106,1655357689000


In [0]:
# Read a dataset in CSV format from S3 to Databricks
# File location and type
file_location = "/mnt/datatraining-csv/yellow_tripdata_2020-05.csv"
file_type = "csv"
# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","
# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)
display(df)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2020-05-01T00:02:28.000+0000,2020-05-01T00:18:07.000+0000,1.0,0.0,1.0,N,234,256,1,12.2,3.0,0.5,2.4,0.0,0.3,18.4,2.5,null
1,2020-05-01T00:23:21.000+0000,2020-05-01T00:26:01.000+0000,2.0,0.4,1.0,N,264,264,1,4.0,0.5,0.5,0.5,0.0,0.3,5.8,0.0,null
1,2020-05-01T00:54:58.000+0000,2020-05-01T00:57:11.000+0000,1.0,0.3,1.0,N,264,264,2,3.5,0.5,0.5,0.0,0.0,0.3,4.8,0.0,null
1,2020-05-01T00:07:10.000+0000,2020-05-01T00:12:46.000+0000,1.0,1.7,1.0,N,107,229,2,7.0,3.0,0.5,0.0,0.0,0.3,10.8,2.5,null
1,2020-05-01T00:55:47.000+0000,2020-05-01T01:01:54.000+0000,0.0,0.9,1.0,N,237,262,1,6.0,3.0,0.5,1.2,0.0,0.3,11.0,2.5,null
1,2020-05-01T00:38:37.000+0000,2020-05-01T01:03:08.000+0000,0.0,12.0,1.0,N,140,28,1,35.5,3.0,0.5,0.0,0.0,0.3,39.3,2.5,null
1,2020-05-01T00:09:46.000+0000,2020-05-01T00:27:44.000+0000,1.0,2.9,1.0,N,79,237,2,12.0,3.0,0.5,0.0,0.0,0.3,15.8,2.5,null
1,2020-05-01T00:22:48.000+0000,2020-05-01T00:28:58.000+0000,1.0,1.5,1.0,N,140,263,1,7.0,3.0,0.5,2.15,0.0,0.3,12.95,2.5,null
1,2020-05-01T00:57:59.000+0000,2020-05-01T01:07:48.000+0000,1.0,2.0,1.0,N,140,230,2,9.0,3.0,0.5,0.0,0.0,0.3,12.8,2.5,null
1,2020-05-01T00:10:53.000+0000,2020-05-01T00:14:17.000+0000,1.0,0.5,1.0,N,239,239,2,4.5,3.0,0.5,0.0,0.0,0.3,8.3,2.5,null
